In [82]:
import yfinance as yf
import pandas as pd
import os
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pickle

cache_file = 'market_cap_cache.csv'

# Function to get market capitalization from Yahoo Finance
def get_market_cap(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    market_cap = stock.info.get('marketCap', None)
    return market_cap

# Check if cache file exists
if os.path.exists(cache_file):
    # Load cached data if it exists
    cached_data = pd.read_csv(cache_file)
    print("Loaded market cap data from cache.")
else:
    # Read the S&P 500 data from the CSV file
    sp500_df = pd.read_csv('sp500_companies_industries.csv')

    # Initialize a list for storing market cap data
    treemap_data = []

    # Fetch the market cap and industries
    for index, row in sp500_df.iterrows():
        ticker = row['Ticker']
        company_name = row['Company']
        industry = row['Industry']
        market_cap = get_market_cap(ticker)

        if market_cap is not None:
            treemap_data.append({
                   'Ticker': ticker,
                'Company': company_name,
                'Industry': industry,
                'MarketCap': market_cap
            })

    # Convert the data into a DataFrame
    cached_data = pd.DataFrame(treemap_data)

    # Save the data to CSV to use as cache
    cached_data.to_csv(cache_file, index=False)

# Calculate total market cap per industry
industry_market_caps = cached_data.groupby('Industry')['MarketCap'].sum().reset_index()
industry_market_caps.columns = ['Industry', 'TotalMarketCap']

# Merge total market cap back into the main DataFrame
treemap_df = pd.merge(cached_data, industry_market_caps, on='Industry')

# Create the treemap plot using Plotly Express with darker colors
def create_treemap():
    fig = px.treemap(
        treemap_df,
        path=[px.Constant("S&P 500"), 'Industry', 'Company'],
        values='MarketCap',
        color='MarketCap',
        color_continuous_scale=[(0.0, '#0d0d0d'), (0.5, '#222222'), (1.0, '#444444')]
    )
    
    fig.update_traces(
        hovertemplate=(
            '<b>Company:</b> %{label}<br>' +
            '<b>Market Cap:</b> $%{value:,.2f}<extra></extra>'
        )
    )
    fig.update_layout(
        plot_bgcolor='#010103',
        paper_bgcolor='#010103',
        font_color='#e6e6e6',
        width=1310,
        height=850,
        showlegend=False,
        margin=dict(l=0, r=0, t=0, b=0)
    )
    return fig


app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY],  meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1"}
    ])

app.layout = dbc.Container([
    dcc.Location(id='url', refresh=False),
    html.Div([
        html.Div([
            html.H1([
                "FinSight",
                html.Br(),
                html.Span("Visualization of Company Financials")
            ], style={'color': '#e6e6e6'}),
            html.P("Search or Click on a Company to view their financials!", style={'color': '#cccccc'})
        ], style={"vertical-align": "top", "height": 210}),
        
        html.Div([
            dbc.RadioItems(
                className='btn-group',
                inputClassName='btn-check',
                labelClassName="btn btn-outline-light",
                labelCheckedClassName="btn btn-light",
                options=[{"label": "Home", "value": 1}, {"label": "Visuals", "value": 2}],
                value=1
            ),
            dcc.Input(
                id="search-input",
                type="text",
                placeholder="Search Company...",
                style={'width': 200, 'backgroundColor': '#333333', 'color': '#e6e6e6'}
            )
        ], style={'display': 'flex', 'align-items': 'center', 'margin-left': 15, "height":220})
    ], style={'width': 340, 'margin-left': 35, 'margin-top': 35}),

    html.Div(id='page-content', style={'width': 890, 'margin': 40})
], fluid=True, style={'display': 'flex', 'backgroundColor': '#010103'})


main_page_layout = html.Div([
    dcc.Graph(id='treemap-graph', figure=create_treemap())
])


# Update the page URL based on clicks on the treemap
@app.callback(
    Output('url', 'pathname'),
    Input('treemap-graph', 'clickData')
)
def update_url(click_data):
    if click_data:
        item_name = click_data['points'][0]['label']
        if item_name in treemap_df['Company'].values:
            # If a company is clicked, navigate to its detail page
            return f'/item/{item_name}'
    return '/'  # Return to the main page if no company is clicked


# Update the page content based on the URL path
@app.callback(Output('page-content', 'children'), Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/' or pathname == '':
        return main_page_layout
    elif pathname.startswith('/item/'):
        item_name = pathname.split('/')[-1]
        return html.Div([
            html.H1(f"Details for {item_name.capitalize()}"),
            html.P("This is a blank page where you can add more information about this item."),
            html.Br(),
            # Button to return to the main page
            html.A(
                html.Button("Back to Treemap", id="back-button"),
                href="/"  # URL to navigate back to the root page
            )
        ])
    return main_page_layout

if __name__ == "__main__":
    app.run_server(debug=True, port=8050)


Loaded market cap data from cache.


In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Sample data
data = {
    "Category": ["Fruit", "Fruit", "Fruit", "Vegetable", "Vegetable", "Vegetable"],
    "Item": ["Apple", "Banana", "Grapes", "Carrot", "Broccoli", "Spinach"],
    "Amount": [500, 300, 200, 150, 100, 80]
}
df = pd.DataFrame(data)

# Initialize the Dash app with suppress_callback_exceptions=True
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Main layout with dcc.Location to handle URL changes and a placeholder for dynamic content
app.layout = html.Div([
    dcc.Location(id='url', refresh=True),  # URL component for redirection
    html.Div(id='page-content'),           # Dynamic content based on the URL
])

# Main page layout with the treemap
main_page_layout = html.Div([
    html.H1("Treemap Example with Plotly and Dash"),
    dcc.Graph(
        id="treemap",
        figure=px.treemap(df, path=["Category", "Item"], values="Amount"),
        clickData=None  # Initialize with no click data
    ),
])

# Callback to update page layout based on URL
@app.callback(Output('page-content', 'children'), [Input('url', 'pathname')])
def display_page(pathname):
    # Display the main page layout
    if pathname == '/' or pathname == '':
        return main_page_layout
    # Display a blank page for each item clicked, with a button to go back
    elif pathname.startswith('/item/'):
        item_name = pathname.split('/')[-1]
        return html.Div([
            html.H1(f"Details for {item_name.capitalize()}"),
            html.P("This is a blank page where you can add more information about this item."),
            html.Br(),
            # Button to return to the main page
            html.A(
                html.Button("Back to Treemap", id="back-button"),
                href="/"  # URL to navigate back to the root page
            )
        ])

# Callback to handle item clicks in the treemap and redirect to a new URL
@app.callback(
    Output('url', 'pathname'),
    Output('treemap', 'figure'),
    Input('treemap', 'clickData')
)
def update_treemap(click_data):
    # Check if an item was clicked
    if click_data:
        item_name = click_data['points'][0]['label']
        if item_name in df['Item'].values:
            # Item clicked, navigate to the item page
            return f'/item/{item_name}', px.treemap(df, path=["Category", "Item"], values="Amount")
        else:
            # Category clicked, zoom into that category
            category_name = click_data['points'][0]['label']
            filtered_df = df[df['Category'] == category_name]
            return '/',''  # No URL change, zoom in within the same page
    return '/', px.treemap(df, path=["Category", "Item"], values="Amount")


# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
